In [1]:
# univariate lstm example
import numpy as np
import pandas as pd
from Minu_Timeseries import timeseries_deposit
from sklearn.preprocessing import MinMaxScaler

Excel_Name = 'ac_statement.xlsx'
Sheet_Name = 'DS_1'
df1 = timeseries_deposit(Excel_Name, Sheet_Name, False)
df2 = df1.reset_index()

In [2]:
#Creating Features based on Date

df2['year']=df2['Date'].dt.year
df2['month']=df2['Date'].dt.month
df2['day']=df2['Date'].dt.day
df2['dayofweek_num']=df2['Date'].dt.dayofweek  
df2['week_of_year']=df2['Date'].dt.weekofyear
df2['quarter'] = df2['Date'].dt.quarter  
df2['days_in_month'] = df2['Date'].dt.days_in_month  
df2['is_month_start'] = df2['Date'].apply(lambda x: 1 if x.is_month_start else 0)
df2['is_month_end'] = df2['Date'].apply(lambda x: 1 if x.is_month_end else 0)
df2['is_quarter_start'] = df2['Date'].apply(lambda x: 1 if x.is_quarter_start else 0)
df2['is_quarter_end'] = df2['Date'].apply(lambda x: 1 if x.is_quarter_end else 0)
df2['is_leap_year'] = df2['Date'].apply(lambda x: 1 if x.is_leap_year else 0)
df2['is_weekend'] = df2['Date'].apply(lambda x: 1 if (x.dayofweek == 6 or x.dayofweek == 5) else 0)
df2['is_sunday'] = df2['Date'].apply(lambda x: 1 if (x.dayofweek == 6) else 0)

#Setting up Quantile values year wise
q95_2015 = df2['Deposit_Amount'][df2['year'] == 2015].quantile(.85)
q05_2015 = df2['Deposit_Amount'][df2['year'] == 2015].quantile(.05)
q95_2016 = df2['Deposit_Amount'][df2['year'] == 2016].quantile(.85)
q05_2016 = df2['Deposit_Amount'][df2['year'] == 2016].quantile(.05)
q95_2017 = df2['Deposit_Amount'][df2['year'] == 2017].quantile(.85)
q05_2017 = df2['Deposit_Amount'][df2['year'] == 2017].quantile(.05)
q95_2018 = df2['Deposit_Amount'][df2['year'] == 2018].quantile(.85)
q05_2018 = df2['Deposit_Amount'][df2['year'] == 2018].quantile(.05)
q95_2019 = df2['Deposit_Amount'][df2['year'] == 2019].quantile(.85)
q05_2019 = df2['Deposit_Amount'][df2['year'] == 2019].quantile(.05)

#Replacing Outlier Values with quantile values
df2.loc[ (df2['year'] == 2015) &  (df2['Deposit_Amount'] <= q05_2015 ),'Deposit_Amount' ]=  q05_2015  
df2.loc[ (df2['year'] == 2015)  & ( df2['Deposit_Amount'] > q95_2015) ,'Deposit_Amount'] =  q95_2015      
df2.loc[(df2['year'] == 2016) & ( df2['Deposit_Amount'] <= q05_2016) ,'Deposit_Amount']=  q05_2016  
df2.loc[(df2['year'] == 2016) &  (df2['Deposit_Amount'] > q95_2016 ),'Deposit_Amount'] =  q95_2016
df2.loc[(df2['year'] == 2017) &  (df2['Deposit_Amount'] <= q05_2017) ,'Deposit_Amount'] =  q05_2017  
df2.loc[(df2['year'] == 2017) &  (df2['Deposit_Amount'] > q95_2017),'Deposit_Amount'] =  q95_2017  
df2.loc[(df2['year'] == 2018) &  (df2['Deposit_Amount'] <= q05_2018) ,'Deposit_Amount'] =  q05_2018  
df2.loc[(df2['year'] == 2018) &  (df2['Deposit_Amount'] > q95_2018),'Deposit_Amount'] =  q95_2018
df2.loc[(df2['year'] == 2019) &  (df2['Deposit_Amount'] <= q05_2019) ,'Deposit_Amount'] =  q05_2019  
df2.loc[(df2['year'] == 2019) &  (df2['Deposit_Amount'] > q95_2019),'Deposit_Amount'] =  q95_2019        

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  import sys


In [3]:
df2.head(2)

,Date,Deposit_Amount,year,month,day,dayofweek_num,week_of_year,quarter,days_in_month,is_month_start,is_month_end,is_quarter_start,is_quarter_end,is_leap_year,is_weekend,is_sunday
0,2015-01-01,2000000.0,2015,1,1,3,1,1,31,1,0,1,0,0,0,0
1,2015-01-02,2462620.0,2015,1,2,4,1,1,31,0,0,0,0,0,0,0


In [4]:
#Creating Lag columns for 30 days
lags = 31
for i in range(lags):
    colname = "lag_"+str(i)
    df2[colname] = df2['Deposit_Amount'].shift(i+1)
    df2.loc[df2[colname].isnull(), colname] = df2['Deposit_Amount']




In [5]:
df2.head(2)

,Date,Deposit_Amount,year,month,day,dayofweek_num,week_of_year,quarter,days_in_month,is_month_start,...,lag_21,lag_22,lag_23,lag_24,lag_25,lag_26,lag_27,lag_28,lag_29,lag_30
0,2015-01-01,2000000.0,2015,1,1,3,1,1,31,1,...,2000000.0,2000000.0,2000000.0,2000000.0,2000000.0,2000000.0,2000000.0,2000000.0,2000000.0,2000000.0
1,2015-01-02,2462620.0,2015,1,2,4,1,1,31,0,...,2462620.0,2462620.0,2462620.0,2462620.0,2462620.0,2462620.0,2462620.0,2462620.0,2462620.0,2462620.0


In [8]:

rolling_windows = [i+1 for i in range(31)]
for i in rolling_windows:
    colname = "rolling_"+str(i)
    df2[colname] = df2['lag_0'].rolling(window=i).mean()
    df2.loc[df2[colname].isnull(), colname] = df2['lag_0']

for i in rolling_windows:
    colname = "rolling_"+str(i) + '_std'
    df2[colname] = df2['lag_0'].rolling(window=i).std()
    df2.loc[df2[colname].isnull(), colname] = df2['lag_0']


In [31]:
listy = [2705390.00000,2231310.00000]
def Average(lst): 
    return sum(lst) / len(lst) 
Average(listy)

2468350.0

In [44]:
(2000000.00000 + 2000000 + 2462620.00000)/3

2154206.6666666665

In [47]:
pd.set_option('display.float_format', lambda x: '%.5f' % x)
df2.loc[:12,['Deposit_Amount','lag_0','rolling_2']]
    #,'lag_0','rolling_0','rolling_0_std']

,Deposit_Amount,lag_0,rolling_2
0,2000000.00000,2000000.00000,2000000.00000
1,2462620.00000,2000000.00000,2000000.00000
2,2948160.00000,2462620.00000,2231310.00000
3,0.00000,2948160.00000,2705390.00000
4,3877320.00000,0.00000,1474080.00000
5,3650750.00000,3877320.00000,1938660.00000
6,3455070.00000,3650750.00000,3764035.00000
7,4550200.00000,3455070.00000,3552910.00000
8,4902100.00000,4550200.00000,4002635.00000
9,5614880.00000,4902100.00000,4726150.00000


In [ ]:
2000000.00000

In [ ]:
    


for i in rolling_windows:
    colname = "rolling_"+str(i)
    df2[colname] = df2['lag_0'].rolling(window=i).mean()
    df2.loc[df2[colname].isnull(), colname] = df2['lag_0']
   
for i in rolling_windows:
    colname = "rolling_"+str(i) + '_std'
    df2[colname] = df2['lag_0'].rolling(window=i).std()
    df2.loc[df2[colname].isnull(), colname] = df2['lag_0']

print(df2.columns)

In [ ]:



#df2['Deposit_Amount']=np.log1p(df2['Deposit_Amount'])


lags = 31
for i in range(lags):
    colname = "lag_"+str(i)
    df2[colname] = df2['Deposit_Amount'].shift(i+1)
    df2.loc[df2[colname].isnull(), colname] = df2['Deposit_Amount']
   

rolling_windows = [i+1 for i in range(31)]

for i in rolling_windows:
    colname = "rolling_"+str(i)
    df2[colname] = df2['lag_0'].rolling(window=i).mean()
    df2.loc[df2[colname].isnull(), colname] = df2['lag_0']
   
for i in rolling_windows:
    colname = "rolling_"+str(i) + '_std'
    df2[colname] = df2['lag_0'].rolling(window=i).std()
    df2.loc[df2[colname].isnull(), colname] = df2['lag_0']

print(df2.columns)

#df2.to_csv("df_new_lagmean.csv")
#
#df2.to_csv("df_new.csv")

excluded = ['Date', 'Deposit_Amount']

myfeatures = [item for item in df2.columns if item not in excluded]

feature_cols = myfeatures

label_col = ['Deposit_Amount']


df_train, df_test = df2[0:1095], df2[1095: ]

params = {}
params['learning_rate'] = 0.03
params['max_depth'] = 4
params['n_estimators'] = 500
params['objective'] = 'reg:squarederror'
params['boosting_type'] = 'gblinear'
params['subsample'] = 0.8
params['random_state'] = 42
params['colsample_bytree']=0.6
params['min_child_weight'] = 3
params['reg_alpha'] = 2.2
params['reg_lambda'] = 2.2


reg = xgb.XGBRegressor(**params)

reg.fit(df_train[feature_cols], df_train[label_col],
        eval_set=[(df_train[feature_cols], df_train[label_col]), (df_test[feature_cols], df_test[label_col])],
        early_stopping_rounds=150,eval_metric = ['rmse'],
       verbose=True)

predictions_lgb = reg.predict(df_test[feature_cols])

predictions_lgb = [item if item > 0 else 0 for item in predictions_lgb]

actual = df_test[label_col]


error = math.sqrt(mean_squared_error(df_test[label_col],predictions_lgb ))


print('Test MSE: %.3f' % error)

reg.feature_importances_

Feature_imp = pd.Series(reg.feature_importances_,index=feature_cols).sort_values(ascending=False)

dfm_xgboost = pd.DataFrame(Feature_imp)
dfm_xgboost.to_csv('xgboosting_importancefinal_60.csv')



df_predict = pd.DataFrame(predictions_lgb,columns = ['predictions'])
#df_predict = pd.DataFrame(np.expm1(predictions_lgb),columns = ['predictions'])

#test_real = np.expm1(df_test[label_col])
#
#error_real = math.sqrt(mean_squared_error(test_real[label_col],df_predict['predictions'] ))
#print('Test MSE: %.3f' % error_real)

final =  df_test[label_col].reset_index().join(df_predict)
final.plot()

final.to_csv("final_predictions_5nov.csv")

mean_absolute_error(df_test[label_col],predictions_lgb)

def mean_absolute_percentage_error(y_true, y_pred):
    """Calculates MAPE given y_true and y_pred"""
    y_true, y_pred = np.array(y_true+1), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mean_absolute_percentage_error(df_test[label_col],predictions_lgb)

#HPT


parameters = {'nthread':[8], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [0.02,.03], #so called `eta` value
              'max_depth': [3,4,5, 6],
              'min_child_weight': [4,5,3],
              'silent': [1],
              'subsample': [0.7,0.8],
              'colsample_bytree': [0.6,0.7],
              'n_estimators': [500,1000]}

%%time
xgb_grid = GridSearchCV(reg,
                        parameters,
                        cv = 2,
                        n_jobs = 32,
                        verbose=True,scoring = 'r2')

xgb_grid.fit(df_train[feature_cols], df_train[label_col])

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)


#Prediction on next 1 month


day_1 = '2019-03-06'

day_n =  '2019-04-06'

new_data = pd.DataFrame({'Date': pd.date_range(start = day_1, end = day_n)})

new_data['year']=new_data['Date'].dt.year
new_data['month']=new_data['Date'].dt.month
new_data['day']=new_data['Date'].dt.day
new_data['dayofweek_num']=new_data['Date'].dt.dayofweek  
new_data['week_of_year']=new_data['Date'].dt.weekofyear
new_data['quarter'] = new_data['Date'].dt.quarter  
new_data['days_in_month'] = new_data['Date'].dt.days_in_month  
new_data['is_month_start'] = new_data['Date'].apply(lambda x: 1 if x.is_month_start else 0)
new_data['is_month_end'] = new_data['Date'].apply(lambda x: 1 if x.is_month_end else 0)

new_data['is_quarter_start'] = new_data['Date'].apply(lambda x: 1 if x.is_quarter_start else 0)
new_data['is_quarter_end'] = new_data['Date'].apply(lambda x: 1 if x.is_quarter_end else 0)

#new_data['is_year_start'] = new_data['Date'].apply(lambda x: 1 if x.is_year_start else 0)
#new_data['is_year_end'] = new_data['Date'].apply(lambda x: 1 if x.is_year_end else 0)
new_data['is_leap_year'] = new_data['Date'].apply(lambda x: 1 if x.is_leap_year else 0)
new_data['is_weekend'] = new_data['Date'].apply(lambda x: 1 if (x.dayofweek == 6 or x.dayofweek == 5) else 0)
new_data['is_sunday'] = new_data['Date'].apply(lambda x: 1 if (x.dayofweek == 6) else 0)


oot = pd.DataFrame(columns = df2.columns)

#oot.loc[0,'lag1'] = df2.loc[len-1, 'lag1']

total_no_fut_pred = new_data.shape[0]

df3 = df2.copy()

for j in range(total_no_fut_pred):
    len = df3.shape[0]
    for i in range(31):
        colname = 'lag_'+str(i)
        print(colname)
        print(len-1-i)
        oot.loc[j,colname] = df3.loc[len-1-i, 'Deposit_Amount'].astype('float64')
        oot[colname] = pd.to_numeric(oot[colname])
   
   
    rolling_windows = [i+1 for i in range(31)]
   
    for i in rolling_windows:
        colname = "rolling_"+str(i)
        print(len-1-i)
        print(len-1)
        oot.loc[j,colname] = float(df3.loc[len-1-i: len-1, 'lag_0'].mean())
        colname1 = "rolling_"+str(i)+ '_std'
        oot.loc[j,colname1] = float(df3.loc[len-1-i: len-1, 'lag_0'].std())
        oot[colname] = pd.to_numeric(oot[colname])
        oot[colname1] = pd.to_numeric(oot[colname1])
       
       
    date_cols = ['year', 'month', 'day', 'dayofweek_num', 'week_of_year',
           'quarter', 'days_in_month', 'is_month_start', 'is_month_end',
           'is_quarter_start', 'is_quarter_end', 'is_leap_year', 'is_weekend',
           'is_sunday']
    for col in date_cols:
        oot.loc[j,col] = new_data.loc[j, col].astype('int64')
        oot[col] = pd.to_numeric(oot[col])
       
    oot.loc[j,'Date'] = new_data.loc[j, 'Date']
    oot['Date'] = pd.to_datetime(oot['Date'])
   
    yhat = reg.predict(oot[feature_cols])
   
    oot.loc[j,'Deposit_Amount'] = yhat[j]
    oot['Deposit_Amount'] = pd.to_numeric(oot['Deposit_Amount'])
   
    new_row = oot.loc[[j]]
    df3 = df3.append(new_row,ignore_index = True)

final_oot =  df2[label_col].reset_index().join(oot)
oot[label_col].plot()

df3[label_col].plot()